In [1]:
import pandas as pd
import datetime 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date

In [3]:
click_data = pd.read_csv('JD_click_data.csv')
deliver_data = pd.read_csv('JD_delivery_data.csv')
inventory_data = pd.read_csv('JD_inventory_data.csv')
network_data = pd.read_csv('JD_network_data.csv')
order_data = pd.read_csv('JD_order_data.csv')
sku_data = pd.read_csv('JD_sku_data.csv')
user_data = pd.read_csv('JD_user_data.csv')
deliver_data.rename(columns = {'type':'delivery_type'}, inplace = True)
order_data.rename(columns = {'type':'order_type'}, inplace = True)




In [4]:
#Joining deliver_data and order_data based on order_ID
delivery_data = pd.merge(deliver_data,order_data, on =['order_ID'],how='right')
print(delivery_data.shape)
delivery_sku_data = pd.merge(delivery_data,sku_data, on =['sku_ID'],how='left')
print(delivery_sku_data.shape)

(550017, 22)
(550017, 28)


In [8]:
delivery_sku_data['same_region'] = 'No'
# Assign value of 'yes' to rows where column A equals column B
delivery_sku_data.loc[delivery_sku_data['dc_ori'] == delivery_sku_data['dc_des'], 'same_region'] = 'yes'

In [28]:
delivery_sku_data_filter = delivery_sku_data.query('promise != "-"')
delivery_sku_data_filter['promise']= delivery_sku_data_filter['promise'].astype(int) *24
delivery_sku_data_filter.shape
delivery_sku_data_filter['actual_delivery_time'] = ((delivery_sku_data_filter['arr_time'].astype('datetime64[ns]') - delivery_sku_data_filter['order_time'].astype('datetime64[ns]'))/ pd.Timedelta(hours=1))

C:\Users\Hannah\AppData\Local\Temp\ipykernel_26952\1847583892.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delivery_sku_data_filter['promise']= delivery_sku_data_filter['promise'].astype(int) *24
C:\Users\Hannah\AppData\Local\Temp\ipykernel_26952\1847583892.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delivery_sku_data_filter['actual_delivery_time'] = ((delivery_sku_data_filter['arr_time'].astype('datetime64[ns]') - delivery_sku_data_filter['order_time'].astype('datetime64[ns]'))/ pd.Timedelta

In [25]:
delivery_sku_data_filter.columns

Index(['package_ID', 'order_ID', 'delivery_type', 'ship_out_time',
       'arr_station_time', 'arr_time', 'user_ID', 'sku_ID', 'order_date',
       'order_time', 'quantity', 'order_type', 'promise',
       'original_unit_price', 'final_unit_price', 'direct_discount_per_unit',
       'quantity_discount_per_unit', 'bundle_discount_per_unit',
       'coupon_discount_per_unit', 'gift_item', 'dc_ori', 'dc_des', 'type',
       'brand_ID', 'attribute1', 'attribute2', 'activate_date',
       'deactivate_date', 'same_region', 'actual_delivery_time'],
      dtype='object')

In [29]:
print(delivery_sku_data_filter.groupby('order_type')['promise'].mean())
print(delivery_sku_data_filter.groupby('order_type')['actual_delivery_time'].mean())

print(delivery_sku_data_filter['order_type'].value_counts())

order_type
1    39.111594
2    79.558971
Name: promise, dtype: float64
order_type
1    29.319685
2    60.469117
Name: actual_delivery_time, dtype: float64
1    274154
2     67279
Name: order_type, dtype: int64


In [31]:
#order level delivery data across region - mean
print('avg promise cross region')
print(delivery_sku_data_filter.query('same_region == "No"').groupby('order_type')['promise'].mean())
print('avg actual cross region')
print(delivery_sku_data_filter.query('same_region == "No"').groupby('order_type')['actual_delivery_time'].mean())
print('number of rows used to calculate')
print(delivery_sku_data_filter.query('same_region == "No"')['order_type'].value_counts())

avg promise cross region
order_type
1    54.895881
2    84.861103
Name: promise, dtype: float64
avg actual cross region
order_type
1    38.795422
2    64.129144
Name: actual_delivery_time, dtype: float64
number of rows used to calculate
1    95055
2    39828
Name: order_type, dtype: int64


In [37]:
#order level delivery data across region - std
print('avg promise cross region')
print(delivery_sku_data_filter.query('same_region == "No"').groupby('order_type')['promise'].std())
print('avg actual cross region')
print(delivery_sku_data_filter.query('same_region == "No"').groupby('order_type')['actual_delivery_time'].std())

avg promise cross region
order_type
1    26.336474
2    34.368742
Name: promise, dtype: float64
avg actual cross region
order_type
1    26.883105
2    37.390717
Name: actual_delivery_time, dtype: float64


In [34]:
# order level delivery data same region - mean
print('avg promise same region')
print(delivery_sku_data_filter.query('same_region == "yes"').groupby('order_type')['promise'].mean())
print('avg actual same region')
print(delivery_sku_data_filter.query('same_region == "yes"').groupby('order_type')['actual_delivery_time'].mean())
print('number of rows used to calculate')
print(delivery_sku_data_filter.query('same_region == "yes"')['order_type'].value_counts())

avg promise same region
order_type
1    30.734242
2    71.866234
Name: promise, dtype: float64
avg actual same region
order_type
1    24.298589
2    55.305135
Name: actual_delivery_time, dtype: float64
number of rows used to calculate
1    179099
2     27451
Name: order_type, dtype: int64


In [38]:
#order level delivery data same region - std
print('avg promise same region')
print(delivery_sku_data_filter.query('same_region == "yes"').groupby('order_type')['promise'].std())
print('avg actual same region')
print(delivery_sku_data_filter.query('same_region == "yes"').groupby('order_type')['actual_delivery_time'].std())

avg promise same region
order_type
1    15.042062
2    34.678143
Name: promise, dtype: float64
avg actual same region
order_type
1    21.860516
2    37.028332
Name: actual_delivery_time, dtype: float64


In [ ]:
order_data
missing_promise_order = order_data.query('promise == "-"')
print(len(missing_promise_order['order_ID'].unique()))
print(len(missing_promise_order.query('dc_ori == dc_des')['order_ID'].unique()))
print(len(missing_promise_order.query('dc_ori != dc_des')['order_ID'].unique()))
# print(missing_promise_order.query('dc_ori != dc_des').shape)

In [ ]:
bins = [0, 100, 200, 300,400,500,600,700,800,900,float('inf')]

# Create labels for each bin
labels = ['0 - 100', '100 - 200', '200 - 300', '300 - 400', '400 - 500', '500 - 600', '600 - 700', '700 - 800', '800 - 900','900 +']

# Add new column to DataFrame with price ranges as categories
delivery_sku_data['Price_Range'] = pd.cut(delivery_sku_data['final_unit_price'], bins=bins, labels=labels)


In [ ]:
delivery_sku_data.groupby(['Price_Range','promise','order_type','same_region'])['order_ID'].count().reset_index().to_csv("order_data1.csv",index=False)
# delivery_sku_data.shape

In [ ]:
delivery_sku_data.query('gift_item == 1').head().T

In [ ]:
delivery_sku_data.head().T

In [ ]:
print(with_promise.query('dc_ori == dc_des and order_type == 1')['Price_Range'].value_counts())

In [ ]:
print(with_promise.query('dc_ori == dc_des and order_type == 2')['Price_Range'].value_counts())

In [ ]:
print(with_promise.query('dc_ori != dc_des and order_type == 1')['Price_Range'].value_counts())

In [ ]:
print(with_promise.query('dc_ori != dc_des and order_type == 2')['Price_Range'].value_counts())

In [ ]:
bins = [0, 100, 200, 300,400,500,600,700,800,900,float('inf')]

# Create labels for each bin
labels = ['0 - 100', '100 - 200', '200 - 300', '300 - 400', '400 - 500', '500 - 600', '600 - 700', '700 - 800', '800 - 900','900 +']

# Add new column to DataFrame with price ranges as categories
missing_promise_order['Price_Range'] = pd.cut(missing_promise_order['final_unit_price'], bins=bins, labels=labels)
# delivery_sku_data_filtered_s['promise_actual_difference'] = delivery_sku_data_filtered_s['promise'] - delivery_sku_data_filtered_s['actual_delivery_time']
print(missing_promise_order['Price_Range'].value_counts())

In [ ]:
missing_promise_order.query('dc_ori == dc_des')['Price_Range'].value_counts()

In [ ]:
missing_promise_order.query('dc_ori != dc_des')['Price_Range'].value_counts()

In [ ]:
missing_promise_order.head().T

In [ ]:
delivery_sku_data['actual_delivery_time'] = ((delivery_sku_data['arr_time'].astype('datetime64[ns]') - delivery_sku_data['order_time'].astype('datetime64[ns]'))/ pd.Timedelta(hours=1))/24
delivery_sku_data = delivery_sku_data.replace('-',0)
delivery_sku_data_filtered = delivery_sku_data.query('promise != "0"')
delivery_sku_data_filtered['same_region'] = 'No'
# Assign value of 'yes' to rows where column A equals column B
delivery_sku_data_filtered.loc[delivery_sku_data_filtered['dc_ori'] == delivery_sku_data_filtered['dc_des'], 'same_region'] = 'yes'
delivery_sku_data_filtered['promise'] =delivery_sku_data_filtered['promise'].astype(int)

In [ ]:
# print(delivery_sku_data.query('actual_delivery_time != "0"').shape)
# print(delivery_sku_data.query('actual_delivery_time == "0"').shape)

delivery_sku_data['promise'].value_counts()

In [ ]:
# Order data where the actual delivey time is less than promise time
delivery_sku_data_filtered_s = delivery_sku_data_filtered.query('promise < actual_delivery_time')
# delivery_sku_data_filtered_s.head().T

In [ ]:
delivery_sku_data_filtered_s.columns

In [ ]:
    
bins = [0, 100, 200, 300,400,500,600,700,800,900,float('inf')]

# Create labels for each bin
labels = ['0 - 100', '100 - 200', '200 - 300', '300 - 400', '400 - 500', '500 - 600', '600 - 700', '700 - 800', '800 - 900','900 +']

# Add new column to DataFrame with price ranges as categories
delivery_sku_data_filtered_s['Price_Range'] = pd.cut(delivery_sku_data_filtered_s['final_unit_price'], bins=bins, labels=labels)
delivery_sku_data_filtered_s['promise_actual_difference'] = delivery_sku_data_filtered_s['actual_delivery_time'] - delivery_sku_data_filtered_s['promise'] 

In [ ]:
delivery_sku_data_filtered_s.groupby(['order_type','same_region','Price_Range','promise'])['promise_actual_difference'].mean().reset_index().to_csv('test2.csv',index=False)

In [ ]:
import os

os.getcwd()